# Kangaroo Dashboard

#### Links

``https://medium.com/plotly/how-to-create-a-beautiful-interactive-dashboard-layout-in-python-with-plotly-dash-a45c57bb2f3c``\
``https://fonts.google.com/?query=poppins&preview.text=poppins.``\
``https://github.com/plotly/datasets/tree/master/dash-layout-tutorial``

## Libraries

In [1]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import random

## Data

In [2]:
df = pd.read_csv('./kangaroo.csv', encoding='utf-8')  

In [3]:
year_list = np.sort(df.Year.unique())
year_list = year_list.astype('str') 
year_list = np.insert(year_list, 0, 'All')

In [4]:
colors_bright =  ['#ff3d44', '#74eafc', '#80f569', '#ffb85c', '#e969f5', '#fcfa62']
#                  red        blue       green      orange     violet     yellow
colors_rainbow = ['red',     'orange',  'yellow',  'green',   'blue',    'purple']
#                  red        orange     yellow     green      blue       violet
colors_pastel = ['#F2C6DE', '#C6DEF1', '#C9E4DE', '#F7D9C4', '#DBCDF0', '#FAEDCB']
#                  red        blue       green      orange     violet     yellow
colors = colors_bright

## App

In [5]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

## Charts

### go.scatter species by year (fig_a)

In [6]:
df_a = df.groupby(['Year']).sum()
df_a.drop('State', axis=1, inplace=True)
df_a.reset_index(inplace=True)
fig_a = go.Figure()
for i in range(len(df_a.columns[1:])):
    fig_a.add_trace(go.Scatter(x = df_a.Year.to_list(), 
                               y = df_a[df_a.columns[1:][i]].to_list(), 
                               opacity = 0.8, 
                               mode = 'lines', 
                               name = df_a.columns[1:][i], 
                               marker = dict(color=colors[i]),
                               line = dict(width=6.0),))
fig_a.update_layout(xaxis_title='Years', 
                    yaxis_title='Population', 
                    template = 'plotly_dark', 
                    width=790, height=730)
print('Created Species Graph')

Created Species Graph


### go.scatter province by year (fig_b)

In [7]:
df_b = df.loc[:]
df_b['Total'] = df_b.sum(axis=1, numeric_only=True)
df_b = df_b[['Year','State','Total']]
df_b = df_b.groupby(['Year', 'State']).mean()
df_b = df_b.pivot_table(values='Total', index='Year', columns='State').reset_index()
df_b.rename_axis(None, axis=1, inplace=True)
fig_b = go.Figure()
for i in range(len(df_b.columns[1:])):
    fig_b.add_trace(go.Scatter(x = df_b.Year.to_list(), 
                               y = df_b[df_b.columns[1:][i]].to_list(), 
                               opacity = 0.8,
                               mode = 'lines', 
                               marker = dict(color=colors[2+i]), 
                               name = df_b.columns[1:][i],
                               line = dict(width=6.0),))
fig_b.update_layout(xaxis_title = 'Years', 
                    yaxis_title='Population', 
                    template = 'plotly_dark', 
                    width=790, height=730)
print('Created Province Graph')

Created Province Graph


### go.Scttergl (star_fig)

In [8]:
star_fig = go.Figure(
    go.Scattergl(
        x = np.random.randn(1000),
        y = np.random.randn(1000),
        mode='markers',
        marker=dict(color=random.sample(['#ecf0f1']*500 + ["#3498db"]*500, 1000), line_width=1)
    )
)
star_fig.update_layout(template = 'plotly_dark', width=790, height=730,
                  xaxis_visible=False, yaxis_visible=False, 
                  showlegend=False, margin=dict(l=0,r=0,t=0,b=0))
print('Created Star Chart')

Created Star Chart


## Callbacks

### Radio

In [9]:
    @app.callback([Output(component_id='radio-out', component_property='children')],
                  [Input(component_id='select-radio', component_property='value')],
                  [State("select-radio","options")])
    
    def display_buttons(value_radio, options_radio):
        label_radio = [opt['label'] for opt in options_radio if opt['value'] == value_radio][0]
        return [html.Div(html.H3(label_radio), className='Output'),]        

### Dropdowns

In [10]:
@app.callback([Output(component_id='option-one', component_property='children'),
               Output(component_id='option-two', component_property='children'),
               Output(component_id='option-three', component_property='children')],
              [Input(component_id='select-one', component_property='value'),
               Input(component_id='select-two', component_property='value'),
               Input(component_id='select-three', component_property='value')],
              [State("select-one","options"),
               State("select-two","options"),
               State("select-three","options")])

def display_option (value_one, value_two, value_three, options_one, options_two, options_three):
    
    label_one =   [opt['label'] for opt in options_one if opt['value'] == value_one][0]
    label_two =   [opt['label'] for opt in options_two if opt['value'] == value_two][0]
    label_three = [opt['label'] for opt in options_three if opt['value'] == value_three][0]
    
    return [html.Div(html.H3(label_one), className='Output'),
            html.Div(html.H3(label_two), className='Output'),
            html.Div(html.H3(label_three), className='Output')]

### Figure

In [11]:
@app.callback([Output(component_id='plot-container', component_property='children')],
              [Input(component_id='select-one', component_property='value'),
               Input(component_id='select-radio', component_property='value')])

def display_star (select_one_value, select_radio_value):

    if select_radio_value == 1:
        if select_one_value == 1: 
            fig = fig_a
        else: 
            fig = fig_b
    else:
        fig = star_fig
    
    return [html.Div(dcc.Graph(figure=fig), style={'display': 'flex'})]

## Layout

In [12]:
app.layout = dbc.Container([
    html.Div([
        
        html.Div([
            html.H1([
                html.Span("Kangaroo"),
                html.Br(),
                html.Span("Population"),
                html.Br(),
                html.Span("Australia")
            ]),
            html.
            P("Analysis of population of four species of kangaroos by Australian province for the years 2001 to 2011")
        ],  style={"vertical-alignment": "top", "height": 260}),
        
        html.Div([
            html.Div(
                dbc.RadioItems(
                    id = 'select-radio',
                    className='btn-group',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-light",
                    labelCheckedClassName="btn btn-light",
                    options=[
                        {"label": "Graph", "value": 1},
                        {"label": "Chart", "value": 2},
                        {"label": "About", "value": 3}
                    ],
                    value=1,
                    style={'width': '100%'}
                ), style={'width': 309}
            ),
        ], style={'margin-left': 15, 'margin-right': 15, 'display': 'flex'}),
        
        html.Div([
            html.Div([
                html.H2('Category'),
                dcc.Dropdown(
                    id='select-one',
                    options=[
                        {'label': 'Species',  'value': 1},
                        {'label': 'Province', 'value': 2}                   
                    ],
                    value=1,
                    clearable=False,
                    optionHeight=40,
                    className='customDropdown'
                )
            ]),
            html.Div([
                html.H2('Palette'),
                dcc.Dropdown(
                    id='select-two',
                    options=[
                        {'label': 'Bright', 'value': 1},
                        {'label': 'Rainbow', 'value': 2},
                        {'label': 'Pastel', 'value': 3}
                    ],
                    value=1,
                    clearable=False,
                    optionHeight=40,
                    className='customDropdown'
                )
            ]),
            html.Div([
                html.H2('Year'),
                dcc.Dropdown(
                    id='select-three',
                    options=[{'label': str(y), 'value': y} for y in year_list],
                    value='All',
                    clearable=False,
                    optionHeight=40,
                    className='customDropdown'
                )
            ])
        ], style={'margin-left': 15, 'margin-right': 15, 'margin-top': 30}),
        
        html.Div(
            html.Img(src='assets/logo6.png', style={'margin-left': 15, 
                                                    'margin-right': 15, 
                                                    'margin-top': 30, 
                                                    'width': 310})
        )
    ], style={
        'width': 340,
        'margin-left': 35,
        'margin-top': 35,
        'margin-bottom': 35
    }),
    html.Div(
        [         
            html.Div(id='plot-container', className='chart-item', style={'width': 790}),
            
            html.Div([       
                html.H2('Type'),
                html.Div(html.H3("Selected Radio"), className='Output', id='radio-out'),
                html.H2('Category'),
                html.Div(html.H3("Selected Value"), className='Output', id='option-one'),
                html.H2('Color'),
                html.Div(html.H3("Selected Value"), className='Output', id='option-two'),
                html.H2('Year'),
                html.Div(html.H3("Selected Value"), className='Output', id='option-three'),      
            ],  style={'width': 198}),
        ],
        style={
            'width': 990,
            'margin-top': 35,
            'margin-right': 35,
            'margin-bottom': 35,
            'display': 'flex'
        })
],
   fluid=True,
   style={'display': 'flex'},
   className='dashboard-container')

## Launch

In [13]:
if __name__ == '__main__':
    app.run_server(port=8502,
                   debug=True,
                   mode='jupyterlab',
                   dev_tools_ui=True,
                   dev_tools_hot_reload=True, 
                   threaded=True)